In [1]:
from __future__ import print_function
from keras.models import model_from_json
from keras.preprocessing import sequence
from lib import words as wd
from lib import embedding as em
from lib import text_extraction as te
import pandas as pd
import json
import re

Using TensorFlow backend.


In [23]:
def is_scadenza(s):
    return re.match(r'.*primo\s+?esercizio.*', s) != None

def post_process_prediction(y_pred, neutral):
    y_post = list(y_pred)
    for i in range(1,len(y_pred)-1):
        y_post[i] = y_post[i+1] if y_post[i] == neutral and y_post[i-1] == y_post[i+1] else y_post[i]
    return y_post

class PartsExtraction(object):
    def __init__(self, keras_model, reduced_dict, labels, maxlen = 100):
        self._model = keras_model
        self._reduced_dict = reduced_dict
        self._labels = labels
        self._maxlen = maxlen
        
    @staticmethod
    def load_from_files(keras_model_filename, keras_weights_filename,
                        reduced_dict_filename, labels=['poteri', 'assemblea', 'clausola', 'non_riconducibile', 'scadenza']):
        with open(keras_model_filename) as f:
            km = model_from_json(f.read())
        km.load_weights(keras_weights_filename)
        with open(reduced_dict_filename) as f:
            rd = json.load(f)
        return PartsExtraction(km, rd, labels)
    
    def _int_sentences(self, sentences):
        splitted_sentences = wd.tokenize_sentences(sentences, min_words=1)
        permitted_words = self._reduced_dict.keys()
        reduced_sentences = list(em.reduce_dictionary(splitted_sentences, permitted_words, min_words=1))
        return [[self._reduced_dict[w] for w in sent] for sent in reduced_sentences]
    
    def extract_parts(self, sentences, post_process=False):
        int_sentences = self._int_sentences(sentences)
        padded_data = sequence.pad_sequences(int_sentences, self._maxlen, padding="pre", truncating="post", value=0, dtype='uint32')
        predictions = self._model.predict(padded_data).argmax(axis=-1)
        for i in range(len(predictions)):
            if is_scadenza(sentences[i]):
                predictions[i] = self._labels.index('scadenza')
                break
        preds = predictions if not post_process else post_process_prediction(predictions, self._labels.index('non_riconducibile'))
        return [self._labels[i] for i in preds]
    
    def extract_parts_dict(self, sentences, predictions=None):
        predictions = self.extract_parts(sentences) if predictions == None else predictions
        df = pd.DataFrame({'sentence':sentences,'prediction':predictions})
        pivoted = df.pivot(columns='prediction', values='sentence')
        return {k:list(filter(None, pivoted[k])) for k in labels} # if k != 'non_riconducibile'
        #return {k:list(filter(None, pivoted[k])) for k in labels if k != 'non_riconducibile'}   
            

In [24]:
labels = ['poteri', 'assemblea', 'clausola', 'non_riconducibile', 'scadenza']

pe = PartsExtraction.load_from_files('models/extraction_model_30.json',
                                     'models/extraction_weights_30.h5',
                                     'first_5000_words_extraction.json')

In [25]:
document = '../extraction/esempi_descrizioni/5115612230001.pdf'#'../extraction/files_to_label/4907913200001.pdf'

txt = te.extract_text(document, do_ocr=False, pages=-1)
sentences = wd.sentences_doc(txt, rep=' ', newline=True)

In [31]:
predictions = pe.extract_parts(sentences, post_process=False)
list(enumerate(predictions))

[(0, 'non_riconducibile'),
 (1, 'non_riconducibile'),
 (2, 'non_riconducibile'),
 (3, 'non_riconducibile'),
 (4, 'non_riconducibile'),
 (5, 'non_riconducibile'),
 (6, 'non_riconducibile'),
 (7, 'non_riconducibile'),
 (8, 'non_riconducibile'),
 (9, 'non_riconducibile'),
 (10, 'non_riconducibile'),
 (11, 'non_riconducibile'),
 (12, 'non_riconducibile'),
 (13, 'non_riconducibile'),
 (14, 'non_riconducibile'),
 (15, 'non_riconducibile'),
 (16, 'non_riconducibile'),
 (17, 'non_riconducibile'),
 (18, 'non_riconducibile'),
 (19, 'non_riconducibile'),
 (20, 'poteri'),
 (21, 'poteri'),
 (22, 'non_riconducibile'),
 (23, 'non_riconducibile'),
 (24, 'poteri'),
 (25, 'scadenza'),
 (26, 'non_riconducibile'),
 (27, 'non_riconducibile'),
 (28, 'non_riconducibile'),
 (29, 'non_riconducibile'),
 (30, 'non_riconducibile'),
 (31, 'non_riconducibile'),
 (32, 'non_riconducibile'),
 (33, 'non_riconducibile'),
 (34, 'non_riconducibile'),
 (35, 'non_riconducibile'),
 (36, 'non_riconducibile'),
 (37, 'non_ricon

In [32]:
sentences[19:23]

[u" E) Il Capitale Sociale \xe8 di Euro 10 000,00 (diecimila e zero centesimi) ed \xe8 sottoscritto interamente dall'unico socio signor Bonet Massimo e versato interamente in data odierna nelle mani del nominando organo amministrativo 1 mediante assegno circolare non trasferibile n 4590445266-04 emesso in data 5 luglio 2017 a favore di detta societ\xe0 dalla Nuova BancaEtruria S p a",
 u" L'organo amministrativo dichiara di aver ricevuto la predetta somma rilasciandone ampia e liberatoria quietanza ed attesta che il capitale sociale \xe8 interamente versato",
 u" F) L'amministrazione della societ\xe0 \xe8 affidata ad un Amministratore Unico che rester\xe0 in carica a tempo indeterminato, fino a revoca o dimissioni",
 u' A svolgere il suddetto ruolo \xe8 chiamata la signora AMBROGETTI SILVIA, nata a Chiusi della Verna (AR) il 7 luglio 1967, residente in Perugia (PG), via Francesca Giostrelli n 10, codice fiscale: MBR SLV 67L47 C663R']

In [33]:
for p,s in zip(predictions, sentences):
    print('\nPREDICTION: {}\n'.format(p))
    print(s)
    


PREDICTION: non_riconducibile

Repertorio n 9116 Raccolta n 4684 COSTITUZIONE DI SOCIETA' A RESPONSABILITA' LIMITATA Repubblica Italiana L'anno duemiladiciassette il giorno sei del mese di luglio in Pistoia e nel mio studio in Corso Silvano Fedi numero 55

PREDICTION: non_riconducibile

 Avanti a me Dottor Nicola Ottavi, Notaio in Pistoia, iscritto al Ruolo del Collegio dei Distretti Notarili Riuniti di Firenze Pistoia e Prato, senza l'assistenza dei testimoni, è presente il signor: - BONET MASSIMO, nato a Perugia (PG) il 9 settembre 1966, residente in Perugia (PG), via Francesca Giostrelli n 10, codice fiscale: BNT MSM 66P09 G478T

PREDICTION: non_riconducibile

 Detto comparente, cittadino italiano, della cui identità personale, io Notaio sono certo, conviene e stipula quanto segue: A) Viene costituita una società a responsabilità limitata sotto la denominazione sociale "B io srl" società unipersonale

PREDICTION: non_riconducibile

 B) La società ha sede nel Comune di Perugia

PRED

In [34]:
#df = pd.DataFrame({'sentence':sentences,'prediction':predictions})
#pivoted = df.pivot(columns='prediction', values='sentence')
#d = {k:list(filter(None, pivoted[k])) for k in labels if k != 'non_riconducibile'}

d = pe.extract_parts_dict(sentences, predictions)

In [35]:
for k,v in d.items():
    print('\n\n'+k.upper().center(127,'.')+'\n')
    print('.\n\n'.join([s.strip() for s in v]))



...........................................................ASSEMBLEA...........................................................

ARTICOLO 6) COMPETENZA DELL'ASSEMBLEA 1 L'assemblea dei soci delibera sulle materie riservate alla competenza dei soci dal presente atto costitutivo, nonché sugli argomenti che uno o più amministratori o tanti soci che rappresentino almeno un terzo del capitale sociale sottopongano alla sua approvazione.

5) la decisione di compiere operazioni che comportano una sostanziale modificazione dell'oggetto sociale o una rilevante modificazione dei diritti dei soci.

ARTICOLO 7) CONVOCAZIONE DELL'ASSEMBLEA 4 1 L'assemblea è convocata dall'organo amministrativo mediante avviso per lettera raccomandata, fax o posta elettronica certificata, spedito ai soci e da essi ricevuto almeno 8 (otto) giorni prima del giorno fissato per l'assemblea.

Nell'avviso di convocazione può già essere contenuta la data per la riunione di un'assemblea di seconda convocazione, che non coi

In [36]:
print(txt)

Repertorio n. 9116
Raccolta n. 4684
COSTITUZIONE DI SOCIETA' A RESPONSABILITA' LIMITATA
Repubblica Italiana
L'anno duemiladiciassette
il giorno sei
del mese di luglio
in Pistoia e nel mio studio in Corso Silvano Fedi numero 55.
Avanti a me Dottor Nicola Ottavi, Notaio in Pistoia, iscritto al Ruolo del Collegio dei
Distretti Notarili Riuniti di Firenze Pistoia e Prato, senza l'assistenza dei testimoni, è
presente il signor:
- BONET MASSIMO, nato a Perugia (PG) il 9 settembre 1966, residente in Perugia
(PG), via Francesca Giostrelli n. 10,
codice fiscale: BNT MSM 66P09 G478T.
Detto comparente, cittadino italiano, della cui identità personale, io Notaio sono
certo, conviene e stipula quanto segue:
A) Viene costituita una società a responsabilità limitata sotto la denominazione
sociale "B.io srl" società unipersonale.
B) La società ha sede nel Comune di Perugia; il socio dichiara che per l'iscrizione
della società nel Registro delle Imprese dovrà essere indicata la sede legale in via
Franc